In [ ]:
import pandas as pd
from keras.layers import LSTM, Input, Bidirectional, TimeDistributed, Dropout, Dense, Activation
from keras.models import Model
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import numpy as np

## Model

In [ ]:
def get_model(input_shape, output_shape, lr = 0.00001):
    inputs = Input(shape=input_shape)
    X = Dropout(0.5)(inputs)
    X = Dense(4096, activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(2048, activation='relu')(X)
    X = Dense(2048, activation='relu')(X)
    X = Bidirectional(LSTM(1024, return_sequences=True))(X)
    outputs = TimeDistributed(Dense(output_shape, activation='softmax'))(X)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=tf.train.AdamOptimizer(lr),
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'],
    )
    
    return model

## Data

In [ ]:
features_train_1 = np.load('./features/res_net/features_train.npy')
features_dev_1 = np.load('./features/res_net/features_dev.npy')

features_train = np.stack([
    features_train_1,
    features_train_1,
], axis=1)

features_dev = np.stack([
    features_dev_1,
    features_dev_1,
], axis=1)

In [ ]:
print('Train shape: ', features_train.shape, 'Dev shape:', features_dev.shape)

In [ ]:
classes_train_1 = np.load('./features/res_net/classes_train_make.npy')
classes_train_1 = to_categorical(classes_train_1)
classes_dev_1 = np.load('./features/res_net/classes_dev_make.npy')
classes_dev_1 = to_categorical(classes_dev_1)
classes_train_2 = np.load('./features/res_net/classes_train_model.npy')
classes_train_2 = to_categorical(classes_train_2)
classes_dev_2 = np.load('./features/res_net/classes_dev_model.npy')
classes_dev_2 = to_categorical(classes_dev_2)

classes_train_1 = np.pad(
    classes_train_1, 
    pad_width=((0, 0), (0, classes_train_2.shape[1] - classes_train_1.shape[1])),
    mode='constant',
    constant_values=((0,0), (0,0)),
)

classes_dev_1 = np.pad(
    classes_dev_1, 
    pad_width=((0, 0), (0, classes_dev_2.shape[1] - classes_dev_1.shape[1])),
    mode='constant',
    constant_values=((0,0), (0,0)),
)

classes_train = np.stack([
    classes_train_1,
    classes_train_2,
], axis=1)

classes_dev = np.stack([
    classes_dev_1,
    classes_dev_2,
], axis=1)

In [ ]:
print('Train classes shape: ', classes_train.shape, 'Dev classes shape:', classes_dev.shape)

## Training

In [ ]:
model = get_model((features_train.shape[1:]), classes_train.shape[2])

In [ ]:
model.summary()

In [ ]:
model.fit(
    features_train,
    classes_train,
    batch_size=128,
    epochs=50,
    validation_data=(features_dev, classes_dev)
)